<a href="https://colab.research.google.com/github/asupraja3/ml-ng-notebooks/blob/main/FeatureEngineering_and_PolynomialRegression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Optional Lab: Feature Engineering and Polynomial Regression

This notebook demonstrates:
- Creating polynomial **features** (e.g., $x, x^2, x^3$)
- Fitting models with **gradient descent**
- Visualizing fits **with** and **without** feature engineering
- Inspecting learned parameters

We use a simple target function: $y = 1 + x^2$.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt

np.set_printoptions(precision=6, suppress=True)


## Helpers: cost, gradient, and gradient descent

In [ ]:

def predict(X, w, b):
    return X @ w + b

def compute_cost(X, y, w, b):
    m = X.shape[0]
    e = X @ w + b - y
    return (e @ e) / (2*m)

def compute_gradient(X, y, w, b):
    m = X.shape[0]
    e = X @ w + b - y
    dj_dw = (X.T @ e) / m
    dj_db = np.sum(e) / m
    return dj_dw, dj_db

def gradient_descent(X, y, w_init=None, b_init=0.0, alpha=1e-2, iters=1000, trace=False):
    n = X.shape[1]
    w = np.zeros(n) if w_init is None else w_init.astype(float).copy()
    b = float(b_init)
    J_hist = []
    tr = []
    for t in range(1, iters+1):
        dj_dw, dj_db = compute_gradient(X, y, w, b)
        w -= alpha * dj_dw
        b -= alpha * dj_db
        J = compute_cost(X, y, w, b)
        J_hist.append(J)
        if trace and t <= 2000:
            tr.append((J, w.copy(), b))
    return w, b, np.array(J_hist), tr


## Data: quadratic target $y = 1 + x^2$

In [ ]:

# Create target data
x = np.arange(0, 20, 1, dtype=float)
y = 1 + x**2
x_lin = x.reshape(-1, 1)  # column vector for linear feature

plt.scatter(x, y, marker='x', c='r', label="Actual Value")
plt.title("Target data: y = 1 + x^2")
plt.xlabel("x"); plt.ylabel("y"); plt.legend(); plt.show()


## 1) No feature engineering (linear model on raw x)

In [ ]:

X = x_lin  # only x
w, b, J_hist, _ = gradient_descent(X, y, alpha=1e-2, iters=1000)

print("No FE → w:", w, "b:", b, " final cost:", J_hist[-1])

plt.scatter(x, y, marker='x', c='r', label="Actual Value")
plt.plot(x, X @ w + b, label="Predicted Value")
plt.xlabel("x"); plt.ylabel("y"); plt.title("No feature engineering"); plt.legend(); plt.show()

for t in range(0, 1001, 100):
    if t == 0:
        print(f"Iteration {t:4d}, Cost: {J_hist[0]:.6e}")
    else:
        print(f"Iteration {t:4d}, Cost: {J_hist[t-1]:.6e}")


## 2) Polynomial feature: replace x with $x^2$

In [ ]:

# Engineer feature: x^2 instead of x
x_sq = (x**2).reshape(-1, 1)
X = x_sq

w2, b2, J_hist2, _ = gradient_descent(X, y, alpha=1e-5, iters=10_000)

print("Using x^2 → w:", w2, "b:", b2, " final cost:", J_hist2[-1])

plt.scatter(x, y, marker='x', c='r', label="Actual Value")
plt.plot(x, X @ w2 + b2, label="Predicted Value")
plt.xlabel("x"); plt.ylabel("y"); plt.title("Added x^2 feature"); plt.legend(); plt.show()


## 3) Polynomial regression with multiple features: $[x, x^2, x^3]$

In [ ]:

# Engineer multiple polynomial features
X = np.c_[x_lin, x**2, x**3]  # shape (m, 3)

w3, b3, J_hist3, _ = gradient_descent(X, y, alpha=1e-7, iters=10_000)

print("Using [x, x^2, x^3] → w:", w3, "b:", b3, " final cost:", J_hist3[-1])

plt.scatter(x, y, marker='x', c='r', label="Actual Value")
plt.plot(x, X @ w3 + b3, label="Predicted Value")
plt.xlabel("x"); plt.ylabel("y"); plt.title("x, x^2, x^3 features"); plt.legend(); plt.show()

# Display polynomial fit explicitly
print("\nModel (approx): y ≈ "
      f"{w3[0]:.4f}·x + {w3[1]:.4f}·x^2 + {w3[2]:.4e}·x^3 + {b3:.4f}")


## 4) Cost curves for each setup

In [ ]:

plt.figure(figsize=(6,4))
plt.plot(J_hist, label="No FE (linear x)")
plt.plot(J_hist2, label="x^2 only")
plt.plot(J_hist3, label="[x, x^2, x^3]")
plt.xlabel("iteration"); plt.ylabel("cost"); plt.title("Cost vs Iteration"); plt.legend(); plt.grid(True, alpha=0.3)
plt.show()
